# I. Import Needed Libraries

Selenium: accesses and controls web browser to submit search and locate web elements by xpath

CSV: reads and writes CSV files

re: regular expression for pattern matching

requests: makes HTTP requests

BeautifulSoup:parses HTML

pandas: data analysis library to manage dataframes


In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv, re, requests
from bs4 import BeautifulSoup
import pandas as pd

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


# II. Instantiate Browser Object 

In [2]:
url = 'https://ww2.amstat.org/meetings/jsm/2017/onlineprogram/'

# chromedriver needs to be in exectuable path or you need to map to where your chromedriver is
browser = webdriver.Chrome()
browser.get(url)


# III. Navigate through webpage and scrape objects

### Creat xpaths for needed items

In [3]:
xpath_search = '//*[@id="ui-accordion-accordion-panel-0"]/table/tbody/tr/td/form/input[3]'
xpath_session = '//*[@id="bottom"]/table/tbody/tr[1]/td/table/tbody/tr[1]/td[1]/a[contains(@href, "SessionID")]'
# '//tbody//tr[1]//td[1]//a[contains(@href, "SessionID")]'
xpath_titles = '//tbody//tr[2]//td//b'
xpath_sponsors = '//tbody//tr[3]//td'
xpath_type = '//tbody//tr/td[4]'

### Click search with empty fields

In [4]:
# wouldn't click on this page so had to execute script instead of standard click() method
element = browser.find_element_by_xpath(xpath_search)
browser.execute_script("arguments[0].click();", element)


# browser.find_element_by_xpath(xpath_search).click()
# wait = WebDriverWait(browser, 30)
# wait.until(EC.presence_of_element_located((By.XPATH, xpath_search)))
# browser.find_element_by_xpath(xpath_search).click()

### Create lists of each field needed

In [9]:
sessions = [item.text for item in browser.find_elements_by_xpath(xpath_session) if 'Session' not in item.text]
titles = [item.text for item in browser.find_elements_by_xpath(xpath_titles)]
sponsors = [item.text.replace('\n', ', ') for item in browser.find_elements_by_xpath(xpath_sponsors) 
            if item.text != 'Sponsor(s)']

session_linkToAbstract = [item.get_attribute('href') for item in browser.find_elements_by_xpath(xpath_session)]

In [10]:
session_type = [item.split(' — ')[1] for item in titles]
titles = [item.split(' — ')[0] for item in titles]

In [11]:
len(sessions) == len(titles) == len(sponsors) == len(session_linkToAbstract) == len(session_type)

True

In [12]:
df = pd.DataFrame({'Session_No' : sessions,
                   'Session_Title' : titles,
                   'Sponsors': sponsors,
                   'Type': session_type,
                   'Session_URL': session_linkToAbstract                   
                  })

In [13]:
df['Session_ID'] = df['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))

In [14]:
df.head(1)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
0,214967,ASA Board of Directors Meeting (Closed),https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA Board of Directors,Other Cmte/Business,214967


In [15]:
df.drop_duplicates(subset = 'Session_URL', inplace = True)
df.reset_index(inplace = True, drop= True)

In [16]:
dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Datasets'
df.to_excel(r'{}\2017Sessions.xlsx'.format(dest))

# Test Code for this year's object locations (xpaths)

In [17]:
df[df['Session_No'].apply(lambda x: bool(re.search(r'(?<!\d)\d{1,3}(?!\d)',x)))]

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID
7,CE_01C,Master the Tidyverse: An Introduction to R for...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Continuing Education ...,214574
8,CE_02C,Longitudinal and Incomplete Data (ADDED FEE),https://ww2.amstat.org/meetings/jsm/2017/onlin...,"ASA , Biometrics Section",Professional Development Continuing Education ...,214043
9,CE_03C,Bayesian Designs for Phase I-II Clinical Trial...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,"ASA , Biometrics Section , Section on Bayesian...",Professional Development Continuing Education ...,214044
10,CE_04C,An Introduction to the Joint Modeling of Longi...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,"ASA , Biometrics Section",Professional Development Continuing Education ...,214046
11,CE_05C,"Bayesian Thinking: Fundamentals, Computation, ...",https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Continuing Education ...,214062
12,CE_06C,Propensity Score Methods and Their Application...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Continuing Education ...,214070
15,CE_07C,Network Meta-Analysis (ADDED FEE),https://ww2.amstat.org/meetings/jsm/2017/onlin...,"ASA , Health Policy Statistics Section",Professional Development Continuing Education ...,214071
16,CE_40P,Preparing Statisticians for Leadership: How to...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Personal Skills Devel...,214612
26,CE_40P,Preparing Statisticians for Leadership: How to...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Personal Skills Devel...,214613
29,CE_01C,Master the Tidyverse: An Introduction to R for...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,ASA,Professional Development Continuing Education ...,214575


In [34]:
test = df['Session_URL'].iloc[49]
test

'https://ww2.amstat.org/meetings/jsm/2017/onlineprogram/ActivityDetails.cfm?SessionID=214441'

In [35]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [39]:
[soup.find_all('tr')[3].find('b').text]

[' 1 ']

# Run Code to extract workshops and abstract_urls

In [40]:
# dataframe of abstracts to scrape with new urls
session_url = []
abstract_url = []
abstract_title = []
session_no_v2 = []

#dataframe of abstracts already written in the session
workshop_abstract = []
workshop_session_no = []
workshop_session_url = []

for link in session_linkToAbstract:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # extract abstract titles and links
    abstract_links = [item.get('href') for item in soup.find_all('a') if re.search('AbstractDetails', str(item.get('href')))]
    
    if len(abstract_links) == 0:
        try:
            abstract_workshop = re.sub(r'(\n|\r|\t)', '', soup.find_all('tr')[6].text)
            id_no = soup.find_all('tr')[3].find('b').text.strip()
            
            workshop_abstract.append(abstract_workshop)
            workshop_session_no.append(id_no)
            workshop_session_url.append(link)
        except:
            continue
    
    else:
        abstract_titles = [item.text for item in soup.find_all('a') 
            if re.search('AbstractDetails', str(item.get('href')))]

        # map back to session id and original link for later joining
        session_ids = [soup.find_all('tr')[3].find('b').text] * len(abstract_links)
        original_link = [link] * len(abstract_links)

        # create list of 
        session_url = session_url + original_link
        abstract_url = abstract_url + abstract_links
        abstract_title = abstract_title + abstract_titles
        session_no_v2 = session_no_v2 + session_ids

    

In [41]:
len(session_url) == len(abstract_url) == len(abstract_title) == len(session_no_v2)

True

In [42]:
len(session_url)

3319

In [43]:
len(workshop_abstract) == len(workshop_session_no) == len(workshop_session_url)

True

In [44]:
len(workshop_abstract)

355

In [45]:
df_abstracts = pd.DataFrame({'Session_URL': session_url,
                            'Abstract_URL' : abstract_url,
                            'Abstract_Title': abstract_title,
                            'Session_No_v2': session_no_v2})

In [46]:
df_abstracts['Session_ID'] = df_abstracts['Session_URL'].apply(lambda x: re.sub(r'https\S+((\d){6})', r'\1', x))
df_abstracts['Abstract_URL'] = df_abstracts['Abstract_URL'].apply(lambda x: url + x)

In [47]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID
0,Asian Initiative: Career Development and Leade...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214819,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214819


In [48]:
df_workshops = pd.DataFrame({'Session_No': workshop_session_no,
                            'Abstract_Text': workshop_abstract,
                            'Session_URL': workshop_session_url})


In [49]:
df_workshops['Session_ID']= df_workshops['Session_URL'].apply(lambda x:re.sub(r'https\S+((\d){6})', r'\1', x))
df_workshops = df_workshops[df_workshops['Abstract_Text'].str.contains('Organizer\(s\)|Chair\(s\)') == False]

In [50]:
df_workshops.head(3)

,Abstract_Text,Session_No,Session_URL,Session_ID
1,,214832,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214832
2,,214856,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214856
3,,214889,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214889


In [51]:
df_abstracts.drop_duplicates(subset = 'Abstract_URL', inplace = True)
df_abstracts.reset_index(drop = True, inplace = True)

In [52]:
df_abstracts.to_excel(r'{}\2017Abstracts.xlsx'.format(dest))
df_workshops.to_excel(r'{}\2017Workshops.xlsx'.format(dest))

# Test extraction of abstract info here

In [60]:
test = df_abstracts['Abstract_URL'].iloc[2]
test

'https://ww2.amstat.org/meetings/jsm/2017/onlineprogram/AbstractDetails.cfm?abstractid=322317'

In [62]:
r = requests.get(test)
soup = BeautifulSoup(r.text, 'lxml')

In [66]:
re.sub(r'\r|\n|\t', '', soup.find_all('tr')[12].find('p').text)

'We introduce a novel functional clustering method, the Bagging Voronoi K-Mediod Aligment (BVKMA) algorithm, which simultaneously clusters and aligns spatially dependent curves. It is a nonparametric statistical method that does not rely on distributional or dependency structure assumptions. The method is motivated by and applied to varved (annually laminated) sediment data from lake Kassjoen in northern Sweden, aiming to infer on past environmental and climate changes. The resulting clusters and their time dynamics show great potential for seasonal climate interpretation, in particular for winter climate changes. '

# Code to extract abstract details from abstract URLs

In [67]:
abs_num = []
abs_keyword = []
abs_type = []
abs_text = []

for link in df_abstracts['Abstract_URL']:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'lxml')
    
    # abstract number
    try:
        number = re.sub(r'\S+abstractid=(\d+)', r'\1', link)
        abs_num.append(number)
    except:
        abs_num.append(None)
    
    # abstract keywords
    try:
        keywords = [item.text for item in soup.find_all('a') if re.search('KeyWordSearch', str(item.get('href')))]
        keywords = ', '.join(keywords)
        abs_keyword.append(keywords)
    except:
        abs_keyword.append(None)
        
    
    # abstract type
    try:
        talk_type = re.sub(r'Type:|\n|\r|\t', '', soup.find_all('tr')[4].text)
        abs_type.append(talk_type)
    except:
        abs_type.append(None)
    
    # abstract text
    try:
        abstract_text = re.sub(r'\r|\n|\t', '', soup.find_all('tr')[12].find('p').text)
        abs_text.append(abstract_text)
    except:
        abs_text.append(None)
    

In [68]:
df_abstracts['Abstract_ID'] = abs_num
df_abstracts['Abstract_Keywords'] = abs_keyword
df_abstracts['Abstract_Type'] = abs_type
df_abstracts['Abstract_Text'] = abs_text

In [69]:
df_abstracts.to_excel(r'{}\2017Abstracts.xlsx'.format(dest))

In [70]:
df_abstracts.head(1)

,Abstract_Title,Abstract_URL,Session_No_v2,Session_URL,Session_ID,Abstract_ID,Abstract_Keywords,Abstract_Type,Abstract_Text
0,Asian Initiative: Career Development and Leade...,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214819,https://ww2.amstat.org/meetings/jsm/2017/onlin...,214819,325513,,Other,"Representatives from IISA, ICSA, KISS, and ASA..."


In [71]:
df_workshops['Abstract_Text'] = df_workshops['Abstract_Text'].apply(lambda x: '' if bool(re.search(r'Chair\(s\):|Organizer\(s\)', x)) else x)



In [72]:
df_details = df_workshops.append(df_abstracts)

In [73]:
df.head(0)

,Session_No,Session_Title,Session_URL,Sponsors,Type,Session_ID


In [74]:
df_final = df_details.merge(df[['Session_Title', 'Sponsors', 'Type', 'Session_ID']], on = 'Session_ID', how= 'outer')

In [75]:
df_final['Unique'] = df_final['Session_ID'].map(str) + df_final['Abstract_ID'].map(str)

In [76]:
df_final.drop_duplicates(subset= 'Unique', inplace = True)
df_final.reset_index(drop = True, inplace = True)

In [77]:
df_final['Session_No'] = df_final['Session_No'].apply(lambda x: x if bool(str(x).startswith('CE') or str(x)[0].isdigit()) else "")

In [78]:
df_final['Session_No_Keep'] = df_final['Session_No_v2'].combine_first(df_final['Session_No'])

In [79]:
del df_final['Session_No']
del df_final['Session_No_v2']

In [80]:
df_final.rename(columns = {'Session_No_Keep': 'Session_No'}, inplace=True)

In [81]:
new_dest = r'C:\Users\Thaunga\Scripts\02. Presentations\JSM\Final Data'
df_final.to_excel(r'{}\2017_JSM_data.xlsx'.format(new_dest))